In [19]:
%pip install tensorflow-macos
%pip install tensorflow-metal


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Primero leemos el archivo csv entregado por nuestros compañeros del TEC 

In [20]:
import pandas as pd

df = pd.read_csv("/Users/S340/Documents/Octavo/Analítica de datos /airbnb-proyecto2-20252/airbnb_limpio.csv")

print("Dimensiones del dataset:", df.shape)
df.head()




Dimensiones del dataset: (5562, 80)


,id,source,name,host_id,host_name,host_since,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,...,has_free_street_parking,has_private_entrance,has_essentials,has_heating,has_wifi,has_pets_allowed,has_hot_water,has_self_check_in,has_freezer,has_exercise_equipment
0,18744501,city scrape,"""Artist´s Creative Residence"" 100m² im Zentrum",129635321,Sylvia,2017-05-09,f,1.0,1.0,"['email', 'phone']",...,1,1,1,1,1,0,1,0,0,0
1,23356842,city scrape,"""Bohemian Residency"" (Central & Quiet) * * * * *",150173398,Vincent,2017-09-11,t,2.0,3.0,"['email', 'phone']",...,1,1,1,1,1,0,1,1,0,0
2,819658084391291386,city scrape,"""Feel at Home"" Flat at the Lerchenauer See",29225873,Skandar,2015-03-12,f,1.0,1.0,"['email', 'phone']",...,0,0,1,1,1,1,1,0,0,1
3,34677963,city scrape,"""Little Star"" Schlafoase im Zentrum",28482431,Adriana,2015-02-28,f,5.0,5.0,"['email', 'phone']",...,0,1,1,1,1,0,1,1,0,0
4,34431776,city scrape,"""Moonlight"" Schlafoase mitten im Szenenviertel",28482431,Adriana,2015-02-28,f,5.0,5.0,"['email', 'phone']",...,0,1,1,1,1,0,1,1,0,0


 ## 2. Selección de variables numéricas y definición de X e y

Para entrenar la red neuronal únicamente utilizamos variables numéricas, por lo cual filtramos el dataset.  
Luego definimos:

- **X** : Variables predictoras  
- **y** : Variable objetivo (`recommended`)



In [ ]:
df["recommended"] = df["host_is_superhost"].map({"t": 1, "f": 0})

df["recommended"].value_counts()

numeric_df = df.select_dtypes(include=["int64", "float64"])

# Incluir también la variable objetivo
numeric_df["recommended"] = df["recommended"]

numeric_df.head()

recommended
0    4329
1    1233
Name: count, dtype: int64

In [23]:
X = numeric_df.drop(columns=["recommended"])
y = numeric_df["recommended"]

## 3. Luego dividimos los datos en entrenamiento y prueba usando train_test_split.
Al aplicar estratificación, nos aseguramos de que la proporción entre superhosts y no-superhosts se mantenga igual en ambos subconjuntos.


In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

((4449, 63), (1113, 63))

## 4. Escalamiento de las variables numéricas

Los modelos basados en redes neuronales requieren que los datos estén en la misma escala.  
Por lo tanto, aplicamos **StandardScaler** para normalizar las variables numéricas.

Usamos:
- `fit_transform`: sobre el conjunto de entrenamiento  
- `transform` : sobre el conjunto de prueba  

Esto evita fugas de información (*data leakage*).

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 5. Construcción del modelo de red neuronal para clasificación

Creamos un modelo secuencial simple utilizando Keras.  
La arquitectura incluye:

- **Capa de entrada** con `input_dim`
- **Dos capas densas ocultas** con activación ReLU  
- **Una capa de salida** con activación sigmoide (para clasificación binaria)

Finalmente se compila con:
- Optimizador: **Adam**  
- Función de pérdida: **binary_crossentropy**  
- Métrica principal: **accuracy**

In [26]:

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Dimensión de entrada (número de columnas de X)
input_dim = X_train_scaled.shape[1]

# Definimos una red neuronal sencilla para clasificación binaria
nn_clf = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(input_dim,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")   # salida entre 0 y 1 (probabilidad de clase 1)
])

nn_clf.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

nn_clf.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,209 (24.25 KB)

 Trainable params: 6,209 (24.25 KB)

 Non-trainable params: 0 (0.00 B)

## 6. Entrenamiento del modelo

Entrenamos la red neuronal durante 50 épocas y con `batch_size = 64`.  
Se utiliza un `validation_split` de 20% para monitorear el desempeño del modelo mientras aprende.

Mostramos:
- Accuracy por época  
- Loss de entrenamiento y validación  
Lo cual nos permite verificar si el modelo converge correctamente.

In [ ]:

history_nn_clf = nn_clf.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    verbose=1
)

Epoch 1/50


2025-12-01 08:45:21.308507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


56/56 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.7137 - loss: 0.6049 - val_accuracy: 0.7820 - val_loss: 0.5445
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8047 - loss: 0.4638 - val_accuracy: 0.7966 - val_loss: 0.5058
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8084 - loss: 0.4315 - val_accuracy: 0.7944 - val_loss: 0.4880
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8089 - loss: 0.4196 - val_accuracy: 0.7933 - val_loss: 0.4843
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8117 - loss: 0.4150 - val_accuracy: 0.8056 - val_loss: 0.4723
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8140 - loss: 0.4155 - val_accuracy: 0.7978 - val_loss: 0.4725
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8162 - loss: 0.4146 - val_accuracy: 0.8011 - val_loss: 0.4613
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8123 - loss: 0.4168 - val_accuracy: 0.7955 - val_loss: 0.5487
Ep

## 7. Evaluación del modelo

Realizamos predicciones sobre el conjunto de prueba.  
Luego evaluamos el modelo con las métricas estándar de clasificación:

- **Accuracy**  
- **Matriz de confusión**  
- **Reporte de clasificación** (precision, recall y F1-score)

Estas métricas permiten entender qué tan bien identifica la clase 1 (superhost) y la clase 0 (no superhost), especialmente considerando el desbalance de clases en el dataset.

In [28]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Predicción de probabilidades (salida sigmoide entre 0 y 1)
y_proba_nn_clf = nn_clf.predict(X_test_scaled)

# Convertimos probabilidades a clases 0/1 con umbral 0.5
y_pred_nn_clf = (y_proba_nn_clf > 0.5).astype("int32")

print("\nAccuracy (NN clasificación):", accuracy_score(y_test, y_pred_nn_clf))

print("\nMatriz de confusión (NN clasificación):")
print(confusion_matrix(y_test, y_pred_nn_clf))

print("\nReporte de clasificación (NN clasificación):")
print(classification_report(y_test, y_pred_nn_clf))

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Accuracy (NN clasificación): 0.8292902066486972

Matriz de confusión (NN clasificación):
[[808  58]
 [132 115]]

Reporte de clasificación (NN clasificación):
              precision    recall  f1-score   support

           0       0.86      0.93      0.89       866
           1       0.66      0.47      0.55       247

    accuracy                           0.83      1113
   macro avg       0.76      0.70      0.72      1113
weighted avg       0.82      0.83      0.82      1113

